<a href="https://colab.research.google.com/github/snpushpi/Differential-Privacy-in-Split-Learning/blob/main/SplitNN_RON_GAUSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import sys
import numpy as np 
import copy

! pip install ipython-autotime
%load_ext autotime

! pip install ipython-autotime
%load_ext autotime
print(f"Python: {sys.version}")
print(f"Pytorch: {torch.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print(f'GPU: {torch.cuda.current_device()}, {torch.cuda.device_count()}, {torch.cuda.get_device_name(0)}, {torch.cuda.is_available()}')
else: print(f'Device: cpu')

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Python: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
Pytorch: 1.6.0+cu101
Device: cpu
time: 2.6 s


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 21.2 s


In [4]:
mnist_data_path = '/content/drive/My Drive/archive'

time: 926 µs


In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))])

# load training set 
mnist_trainset = torchvision.datasets.MNIST(mnist_data_path, train=True, transform=transform, download=True)
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, shuffle=True)

# load test set
mnist_testset = torchvision.datasets.MNIST(mnist_data_path, train=False, transform=transform, download=True)
mnist_test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=128, shuffle=True)

from torch.utils.data.sampler import SubsetRandomSampler

total_size = len(mnist_trainset)

split1 = total_size // 4
split2 = split1*2
split3 = split1*3

indices = list(range(total_size))

alice_idx = indices[:split1]
bob_idx = indices[split1:split2]
mike_idx = indices[split2:split3]
rose_idc = indices[split3:]

alice_sampler = SubsetRandomSampler(alice_idx)
bob_sampler = SubsetRandomSampler(bob_idx)
mike_sampler = SubsetRandomSampler(mike_idx)
rose_sampler = SubsetRandomSampler(rose_idc)


alice_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=alice_sampler)
bob_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=bob_sampler)
mike_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=mike_sampler)
rose_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=rose_sampler)

data_loaders = [alice_loader, bob_loader, mike_loader, rose_loader ]

print(f'Data at alice: {len(alice_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at bob: {len(bob_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at mike: {len(mike_sampler)} \t Batches: {len(mike_loader)}')
print(f'Data at rose: {len(rose_sampler)} \t Batches: {len(rose_loader)}')

Extracting /content/drive/My Drive/archive/MNIST/raw/train-images-idx3-ubyte.gz to /content/drive/My Drive/archive/MNIST/raw


Extracting /content/drive/My Drive/archive/MNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/My Drive/archive/MNIST/raw


Extracting /content/drive/My Drive/archive/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/My Drive/archive/MNIST/raw


Extracting /content/drive/My Drive/archive/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/My Drive/archive/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!
Data at alice: 15000 	 Batches: 118
Data at bob: 15000 	 Batches: 118
Data at mike: 15000 	 Batches: 118
Data at rose: 15000 	 Batches: 118
time: 2.26 s


In [81]:
client_model = torch.nn.Sequential(
                torch.nn.Conv2d(1, 32, kernel_size=5, padding=0, stride=1),  
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
                torch.nn.Conv2d(32, 32, kernel_size=5, padding=0, stride=1),  
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
                )

# send a copy to each data holder
alice_model = copy.deepcopy(client_model)
bob_model = copy.deepcopy(client_model)
mike_model = copy.deepcopy(client_model)
rose_model = copy.deepcopy(client_model)

# keep server copy at Server
server_model = torch.nn.Sequential(
                torch.nn.Flatten(),
                torch.nn.Linear(400, 256),
                torch.nn.ReLU(),
                torch.nn.Linear(256, 10),  
                # torch.nn.Softmax(dim=-1),
        )


models = [alice_model, bob_model, mike_model, rose_model]

# Create optimizers for each model
optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]

#def cor_loss(x, split):
#  retun value

#def total_loss(cor_loss, celoss):
#  return x*cor_loss + y*celoss

time: 48.6 ms


In [18]:
server_optimizer = optim.Adam(server_model.parameters(), lr=0.01)
server_loss = nn.CrossEntropyLoss()

time: 1.96 ms


In [19]:
import numpy as np
from sklearn.preprocessing import normalize
import math


time: 1.29 ms


In [43]:
def pre_processing(dataset,epsilon_mu):
    '''Input Numpy Matrix of dimension (m,n) and privacy param epsion '''
    pre_normalized_dataset = normalize(dataset,axis=0)
    #deriving dp mean
    avg = np.mean(pre_normalized_dataset,axis=1)
  
    m,n = pre_normalized_dataset.shape
    avg = avg.reshape((m,1))
    loc_param = 2*math.sqrt(m)/(n*epsilon_mu)
    laplace_noise = np.random.laplace(0,loc_param,(m,1))
    mu_dp = avg+laplace_noise
    #centerize the data
    mat_1 = np.ones((1,n))
    centralized_dataset = dataset - np.matmul(mu_dp,mat_1)
    final_dataset = normalize(centralized_dataset,axis=0)
    return final_dataset,mu_dp


time: 8.26 ms


In [48]:
def ron_projection(pre_processed_dataset,p):
    m,n = pre_processed_dataset.shape
    A = np.random.uniform(0,1,(m,m))
    Q,R = np.linalg.qr(A, mode='complete')
    #constructing a Ron Projection Matrix
    W = Q[:,:p]
    projected_data = np.matmul(np.transpose(W),pre_processed_dataset)
    return projected_data,W


time: 4.64 ms


In [72]:
def ron_gauss_for_supervised(dataset,training_label,a,dimension_p,epsilon_mu,epsilon_sigma):
    #label.shape = (n,1)
    preprocessed_data,dp_mu = pre_processing(dataset,epsilon_mu)
    ron_projected_data,W = ron_projection(preprocessed_data,dimension_p)
    p,n = ron_projected_data.shape
    augmented_ron = np.vstack((ron_projected_data,np.transpose(training_label)))
    sqp = math.sqrt(p)
    loc_param = (2*sqp+4*a*sqp+a*a)/(n*epsilon_sigma)
    laplace_noise = np.random.laplace(0,loc_param,(dimension_p+1,dimension_p+1))
    dp_cov = (1/n)*np.matmul(augmented_ron,np.transpose(augmented_ron))+laplace_noise
    synthesized_dp_data = np.random.multivariate_normal(np.zeros((dimension_p+1,)), dp_cov, size=(n,))
    return np.transpose(synthesized_dp_data)


time: 5.9 ms


In [87]:
epochs = 7

for e in range(epochs):
  running_loss = 0

  # iterate based on batch numbers assuming it is unified acorss all clients
  # a more efficient solution is to assign the batch size at each client relative on its data size
  # this would guarantee less latency at the server side given that client-side training is parallaized 

  for i in range(len(data_loaders[0])):
    # assuming a gprc setup 
    for opt in optimizers:
      opt.zero_grad()
    
    server_optimizer.zero_grad()

    lst_of_vars = []

    for i in range(len(data_loaders)): 
      imgs, lbls = next(iter(data_loaders[i])) 
      split_output = models[i](imgs) 
      ron_gauss_split_output = split_output.reshape((128,512))
      #cor_loss = (imgs, split_layer_output)
      #we will apply ron gauss projection here
      ron_gauss_dataset = torch.transpose(ron_gauss_split_output,0,1).detach().numpy()
      ron_gauss_labels = lbls.reshape((128,1)).numpy()
      synthesized_dp_data = ron_gauss_for_supervised(ron_gauss_dataset,ron_gauss_labels,10,400,0.6,1.4)
      split_layer_output = torch.from_numpy(np.transpose(synthesized_dp_data[:-1]))
      split_layer_output = split_layer_output.clone().detach().requires_grad_(True)
      #lbls = torch.from_numpy(synthesized_dp_data[-1]).reshape((128,))
      server_output = server_model(split_layer_output.float())
      loss = server_loss(server_output, lbls.long())
      #total = toal()
      lst_of_vars.append({'split_output': split_output, 'split_layer_output': split_layer_output, 'loss':loss})
      
    loss = 0  
    for i in range(len(data_loaders)):
      loss += lst_of_vars[i]['loss']

    avg_loss = loss / len(data_loaders)
    running_loss += avg_loss

    avg_loss.backward()

    for i in range(len(data_loaders)):
      g_split_gradients = lst_of_vars[i]['split_layer_output'].grad.clone().detach()
      lst_of_vars[i]['split_output'].backward(split_gradients)
    
    server_optimizer.step()
    
    for opt in optimizers:
      opt.step()

  print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(data_loaders[0])))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not p

Epoch 1 - Training loss: 5.289231777191162


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not p

Epoch 2 - Training loss: 5.29841423034668


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not p

...

Epoch 5 - Training loss: 5.292015075683594


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not p

Epoch 6 - Training loss: 5.296594619750977


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: covariance is not p

Epoch 7 - Training loss: 5.285242557525635
time: 15min 58s
